In [1]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window
import pandas as pd

In [2]:
list=[[{"value": {"employee": {"employeeid": "1234","employeename": "ABCD","contactNumber": [{"type": "Work","phoneNumber": "1234567890"},{"type": "Home","phoneNumber": "0987654321"}]}}}]]

df=spark.createDataFrame(list,['json'])
df.printSchema()

root
-- json: map (nullable = true)
 |-- key: string
 |-- value: map (valueContainsNull = true)
 | |-- key: string
 | |-- value: map (valueContainsNull = true)
 | | |-- key: string
 | | |-- value: string (valueContainsNull = true)

In [3]:
js=[[{
        "value": {
            "employee": {
                 "employeeid": "1234",
                "employeename": "ABCD",
                 "contactNumber": [
                            {
                                "type": "Work",

                                "phoneNumber": "1234567890"
                            },
                            {
                                "type": "Home",

                                "phoneNumber": "0987654321"
                            }
                        ] }}}]]

df=spark.createDataFrame(js,['json'])
df.show(truncate=False)
df.printSchema()

+----------------------------------------------------------------------------------------------------------------------------------------------------------------+
json |
+----------------------------------------------------------------------------------------------------------------------------------------------------------------+
[value -> [employee -> [contactNumber -> [{type=Work, phoneNumber=1234567890}, {type=Home, phoneNumber=0987654321}], employeeid -> 1234, employeename -> ABCD]]]|
+----------------------------------------------------------------------------------------------------------------------------------------------------------------+

root
-- json: map (nullable = true)
 |-- key: string
 |-- value: map (valueContainsNull = true)
 | |-- key: string
 | |-- value: map (valueContainsNull = true)
 | | |-- key: string
 | | |-- value: string (valueContainsNull = true)

In [4]:
df.select("json.value.employee").show()

+--------------------+
 employee|
+--------------------+
[contactNumber ->...|
+--------------------+

In [5]:
"`value.employee.employeeid`","string", "employeeid","string"),("`value.employee.employeename`","string", "employeename","string"), ("`value.employee.employeename.contactNumber.value.type`","string", "type","string"),("`value.employee.employeename.contactNumber.value.phoneNumber`","string", "phoneNumber","string")

In [6]:
js=[[{"value": {
            "employee": {
                 "employeeid": "1234",
                "employeename": "ABCD",
                 "contactNumber": [
                            {
                                "type": "Work",

                                "phoneNumber": "1234567890"
                            },
                            {
                                "type": "Home",

                                "phoneNumber": "0987654321"
                            }
                        ] }}}]]

df=spark.createDataFrame(js,['json'])

df.show()
df.printSchema()

#+----------------------------------------------------------------------------------------------------------------------------------------------------------------+
#|json                                                                                                                                                            |
#+----------------------------------------------------------------------------------------------------------------------------------------------------------------+
#|[value -> [employee -> [contactNumber -> [{type=Work, phoneNumber=1234567890}, {type=Home, phoneNumber=0987654321}], employeeid -> 1234, employeename -> ABCD]]]|
#+----------------------------------------------------------------------------------------------------------------------------------------------------------------+

#root
 #|-- json: map (nullable = true)
 #|    |-- key: string
 #|    |-- value: map (valueContainsNull = true)
 #|    |    |-- key: string
 #|    |    |-- value: map (valueContainsNull = true)
 #|    |    |    |-- key: string
 #|    |    |    |-- value: string (valueContainsNull = true)

from pyspark.sql.types import *
from pyspark.sql import functions as F

schema = ArrayType(MapType(StringType(),StringType()))
df.select("json.value.employee.employeeid","json.value.employee.employeename",\
         "json.value.employee.contactNumber")\
  .withColumn("contactNumber",F.explode(F.from_json(F.regexp_replace\
                                          (F.regexp_replace("contactNumber","([\\w-]+)", "\"$1\"")\
                                           ,"\=",":"),schema)))\
  .select("employeeid","employeename","contactNumber.type","contactNumber.phoneNumber")\
  .show(truncate=False)

#+----------+------------+----+-----------+
#|employeeid|employeename|type|phoneNumber|
#+----------+------------+----+-----------+
#|      1234|        ABCD|Work| 1234567890|
#|      1234|        ABCD|Home| 0987654321|
#+----------+------------+----+-----------+  

+----------+------------+----+-----------+
employeeid|employeename|type|phoneNumber|
+----------+------------+----+-----------+
 1234| ABCD|Work| 1234567890|
 1234| ABCD|Home| 0987654321|
+----------+------------+----+-----------+

In [7]:
from pyspark.sql.types import *
from pyspark.sql import functions as F

schema = ArrayType(MapType(StringType(),StringType()))
df.select("json.value.employee.employeeid","json.value.employee.employeename",\
         "json.value.employee.contactNumber")\
  .withColumn("contactNumber",F.explode(F.from_json(F.regexp_replace\
                                          (F.regexp_replace("contactNumber","([\\w-]+)", "\"$1\"")\
                                           ,"\=",":"),schema)))\
  .select("employeeid","employeename","contactNumber.type","contactNumber.phoneNumber")\
  .show(truncate=False)

+----------+------------+----+-----------+
employeeid|employeename|type|phoneNumber|
+----------+------------+----+-----------+
1234 |ABCD |Work|1234567890 |
1234 |ABCD |Home|0987654321 |
+----------+------------+----+-----------+

In [8]:
from pyspark.sql.types import *
from pyspark.sql import functions as F

schema = ArrayType(MapType(StringType(),StringType()))
df.select("json.value.employee.employeeid","json.value.employee.employeename",\
         "json.value.employee.contactNumber")\
  .withColumn("contactNumber",F.explode(F.from_json(F.regexp_replace\
                                          (F.regexp_replace("contactNumber","([\\w-]+)", "\"$1\"")\
                                           ,"\=",":"),schema)))\
  .select("employeeid","employeename","contactNumber.type","contactNumber.phoneNumber")\
  .show(truncate=False)


+----------+------------+----+-----------+
employeeid|employeename|type|phoneNumber|
+----------+------------+----+-----------+
1234 |ABCD |Work|1234567890 |
1234 |ABCD |Home|0987654321 |
+----------+------------+----+-----------+

In [9]:
df.select(F.explode("json.value.employee.contactNumber")).show()

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 

 Py4JJavaError : An error occurred while calling o5069.select.
: org.apache.spark.sql.AnalysisException: cannot resolve 'explode(`json`['value']['employee']['contactNumber'])' due to data type mismatch: input to function explode should be array or map type, not string;;
'Project [explode(json#634[value][employee][contactNumber]) AS List()]
+- LogicalRDD [json#634], false

	at org.apache.spark.sql.catalyst.analysis.package$AnalysisErrorAt.failAnalysis(package.scala:42)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis$$anonfun$checkAnalysis$1$$anonfun$apply$3.applyOrElse(CheckAnalysis.scala:126)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis$$anonfun$checkAnalysis$1$$anonfun$apply$3.applyOrElse(CheckAnalysis.scala:111)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$6.apply(TreeNode.scala:303)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$6.apply(TreeNode.scala:303)
	at org.apache.spark.sql.catalyst.trees.CurrentOrigin$.withOrigin(TreeNode.scala:76)
	at org.apache.spark.sql.catalyst.trees.TreeNode.transformUp(TreeNode.scala:302)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$5.apply(TreeNode.scala:300)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$5.apply(TreeNode.scala:300)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$8.apply(TreeNode.scala:353)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapProductIterator(TreeNode.scala:207)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapChildren(TreeNode.scala:351)
	at org.apache.spark.sql.catalyst.trees.TreeNode.transformUp(TreeNode.scala:300)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$transformExpressionsUp$1.apply(QueryPlan.scala:94)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$transformExpressionsUp$1.apply(QueryPlan.scala:94)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$3.apply(QueryPlan.scala:106)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$3.apply(QueryPlan.scala:106)
	at org.apache.spark.sql.catalyst.trees.CurrentOrigin$.withOrigin(TreeNode.scala:76)
	at org.apache.spark.sql.catalyst.plans.QueryPlan.transformExpression$1(QueryPlan.scala:105)
	at org.apache.spark.sql.catalyst.plans.QueryPlan.org$apache$spark$sql$catalyst$plans$QueryPlan$$recursiveTransform$1(QueryPlan.scala:117)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$org$apache$spark$sql$catalyst$plans$QueryPlan$$recursiveTransform$1$2.apply(QueryPlan.scala:122)
	at scala.collection.TraversableLike$$anonfun$map$1.apply(TraversableLike.scala:234)
	at scala.collection.TraversableLike$$anonfun$map$1.apply(TraversableLike.scala:234)
	at scala.collection.mutable.ResizableArray$class.foreach(ResizableArray.scala:59)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:48)
	at scala.collection.TraversableLike$class.map(TraversableLike.scala:234)
	at scala.collection.AbstractTraversable.map(Traversable.scala:104)
	at org.apache.spark.sql.catalyst.plans.QueryPlan.org$apache$spark$sql$catalyst$plans$QueryPlan$$recursiveTransform$1(QueryPlan.scala:122)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$4.apply(QueryPlan.scala:127)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapProductIterator(TreeNode.scala:207)
	at org.apache.spark.sql.catalyst.plans.QueryPlan.mapExpressions(QueryPlan.scala:127)
	at org.apache.spark.sql.catalyst.plans.QueryPlan.transformExpressionsUp(QueryPlan.scala:94)
	at org.apache.spark.sql.catalyst.analysis.C

In [10]:
dictmale={' Bariatrics ': 13, ' Cardiovascular / Pulmonary': 347, ' Neurology': 198}
dictfemale=[{' Bariatrics ': 13, ' Cardiovascular / Pulmonary': 347, ' Neurology': 198}]

In [11]:
list=[[{"value": {
            "employee": {
                 "employeeid": "1234",
                "employeename": "ABCD",
                 "contactNumber": """[{"type": "Work","phoneNumber": "1234567890"},{"type": "Home","phoneNumber": "0987654321"}]"""}}}]]

df=spark.createDataFrame(list,['json'])
df.show()
df.printSchema()

+--------------------+
 json|
+--------------------+
[value -> [employ...|
+--------------------+

root
-- json: map (nullable = true)
 |-- key: string
 |-- value: map (valueContainsNull = true)
 | |-- key: string
 | |-- value: map (valueContainsNull = true)
 | | |-- key: string
 | | |-- value: string (valueContainsNull = true)

In [12]:
from pyspark.sql import functions as F
from pyspark.sql.types import *

schema = ArrayType(MapType(StringType(),StringType()))
df.select(F.explode("json").alias("key","value"))\
  .select("value.employee.employeeid","value.employee.employeename",\
          ("value.employee.contactNumber"))\
  .withColumn("contactNumber", F.explode(F.from_json("contactNumber",schema)))\
  .select("employeeid","employeename","contactNumber.type", "contactNumber.phoneNumber").show()

+----------+------------+----+-----------+
employeeid|employeename|type|phoneNumber|
+----------+------------+----+-----------+
 1234| ABCD|Work| 1234567890|
 1234| ABCD|Home| 0987654321|
+----------+------------+----+-----------+

In [13]:
c= spark.createDataFrame(
  [
    ('Male', [dictmale]),
    ("Female",dictfemale),

  ],
  ['Gender',"medical_specialty"]
)

In [14]:
c.show(truncate=False)

+------+-----------------------------------------------------------------------------+
Gender|medical_specialty |
+------+-----------------------------------------------------------------------------+
Male |[[ Bariatrics -> 13, Cardiovascular / Pulmonary -> 347, Neurology -> 198]]|
Female|[[ Bariatrics -> 13, Cardiovascular / Pulmonary -> 347, Neurology -> 198]]|
+------+-----------------------------------------------------------------------------+

In [15]:
list=[[1,'John',2],
      [2,'Maria',4],
      [3,'Charles',6]]

df=spark.createDataFrame(list,['ID','Name','Support'])


df.show()

+---+-------+-------+
 ID| Name|Support|
+---+-------+-------+
 1| John| 2|
 2| Maria| 4|
 3|Charles| 6|
+---+-------+-------+

In [16]:
df.withColumn("Name")

In [17]:
from pyspark.sql import functions as F
df.withColumn("Name", F.explode(F.expr("""array_repeat(Name,int(Support))"""))).show()

+---+-------+-------+
 ID| Name|Support|
+---+-------+-------+
 1| John| 2|
 1| John| 2|
 2| Maria| 4|
 2| Maria| 4|
 2| Maria| 4|
 2| Maria| 4|
 3|Charles| 6|
 3|Charles| 6|
 3|Charles| 6|
 3|Charles| 6|
 3|Charles| 6|
 3|Charles| 6|
+---+-------+-------+

In [18]:
df.withColumn("Name", F.explode(F.array(*[['Name']*(F.col("Support"))]))).show()

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-4127867316094446> in <module> 
 ----> 1 df . withColumn ( "Name" , F . explode ( F . array ( * [ [ 'Name' ] * ( F . col ( "Support" ) ) ] ) ) ) . show ( ) 

 /databricks/spark/python/pyspark/sql/column.py in _ (self, other) 
 113 def _ ( self , other ) : 
 114 jc = other . _jc if isinstance ( other , Column ) else other
 --> 115 njc = getattr ( self . _jc , name ) ( jc ) 
 116 return Column ( njc ) 
 117 _ . __doc__ = doc

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1255 answer = self . gateway_client . send_command ( command ) 
 1256 return_value = get_return_value(
 -> 1257 answer, self.gateway_client, self.target_id, self.name)
 1258 
 1259 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 61 def deco ( * a , ** kw ) : 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 
 65 s = e . java_exception . toString ( ) 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 
 330 raise Py4JError(

 Py4JJavaError : An error occurred while calling o624.multiply.
: java.lang.RuntimeException: Unsupported literal type class java.util.ArrayList [Name]
	at org.apache.spark.sql.catalyst.expressions.Literal$.apply(literals.scala:79)
	at org.apache.spark.sql.catalyst.expressions.Literal$$anonfun$create$2.apply(literals.scala:168)
	at org.apache.spark.sql.catalyst.expressions.Literal$$anonfun$create$2.apply(literals.scala:168)
	at scala.util.Try.getOrElse(Try.scala:79)
	at org.apache.spark.sql.catalyst.expressions.Literal$.create(literals.scala:167)
	at org.apache.spark.sql.functions$.typedLit(functions.scala:128)
	at org.apache.spark.sql.functions$.lit(functions.scala:111)
	at org.apache.spark.sql.Column.$times(Column.scala:719)
	at org.apache.spark.sql.Column.multiply(Column.scala:734)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)

In [19]:
df.withColumn("Name", F.explode(F.expr("""array(Name)*Support)"""))).show()

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 

 Py4JJavaError : An error occurred while calling z:org.apache.spark.sql.functions.expr.
: org.apache.spark.sql.catalyst.parser.ParseException: 
extraneous input ')' expecting <EOF>(line 1, pos 19)

== SQL ==
array(Name)*Support)
-------------------^^^

	at org.apache.spark.sql.catalyst.parser.ParseException.withCommand(ParseDriver.scala:241)
	at org.apache.spark.sql.catalyst.parser.AbstractSqlParser.parse(ParseDriver.scala:117)
	at org.apache.spark.sql.execution.SparkSqlParser.parse(SparkSqlParser.scala:55)
	at org.apache.spark.sql.catalyst.parser.AbstractSqlParser.parseExpression(ParseDriver.scala:44)
	at com.databricks.sql.parser.DatabricksSqlParser.parseExpression(DatabricksSqlParser.scala:46)
	at org.apache.spark.sql.functions$.expr(functions.scala:1366)
	at org.apache.spark.sql.functions.expr(functions.scala)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)


During handling of the above exception, another exception occurred:

 ParseException Traceback (most recent call last)
 <command-4127867316094445> in <module> 
 ----> 1 df . withColumn ( "Name" , F . explode ( F . expr ( """array(Name)*Support)""" ) ) ) . show ( ) 

 /databricks/spark/python/pyspark/sql/functions.py in expr (str) 
 673 """
 674 sc = SparkContext . _active_spark_context
 --> 675 return Column ( sc . _jvm . functions . expr ( str ) ) 
 676 
 677 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1255 answer = self . gateway_client . send_command ( command ) 
 1256 return_value = get_return_value(
 -> 1257 answer, self.gateway_client, self.target_id, self.name)
 1258 
 1259 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 71 raise AnalysisException ( s . split ( ': ' , 1 ) [ 1 ] , stackTrace ) 
 72 if s . startswith ( 'org.apache.spark.sql.catalyst.parser.ParseException: ' ) : 
 ---> 73 raise ParseException ( s . split ( ': ' , 1 ) [ 1 ] , stackTrace ) 
 74 if s . startswith ( 'org.apache.spark.sql.streaming.StreamingQueryException: ' ) : 
 75 raise StreamingQueryException ( s . split ( ': ' , 1 ) [ 1 ] , stackTrace ) 

 ParseException : "\nextraneous input ')' expecting <EOF>(line 1, pos 19)\n\n== SQL ==\narray(Name)*Support)\n-------------------^^^\n"

In [20]:
df.withColumn("Name", F.expr("""repeat(concat(Name,','),Support)"""))\
  .withColumn("Name", F.explode(F.expr("""split(substring(Name,1,length(Name)-1),',')"""))).show()

+---+-------+-------+
 ID| Name|Support|
+---+-------+-------+
 1| John| 2|
 1| John| 2|
 2| Maria| 4|
 2| Maria| 4|
 2| Maria| 4|
 2| Maria| 4|
 3|Charles| 6|
 3|Charles| 6|
 3|Charles| 6|
 3|Charles| 6|
 3|Charles| 6|
 3|Charles| 6|
+---+-------+-------+

In [21]:
df.withColumn("Name", F.expr("""repeat(concat(Name,','),Support)"""))\
  .withColumn("Name", F.explode(F.expr("""split(substring(Name,1,length(Name)-1),',')"""))).show()

In [22]:
list=[['1'],
      ['2'],
      ['", Beatrice'],
      ['123yo']]
df=spark.createDataFrame(list,['_c0'])

df.show()

+-----------+
 _c0|
+-----------+
 1|
 2|
", Beatrice|
 123yo|
+-----------+

In [23]:
df.withColumn("yo",F.regexp_extract("_c0","\D+",0)).show()

+-----------+-----------+
 _c0| yo|
+-----------+-----------+
 1| |
 2| |
", Beatrice|", Beatrice|
 123yo| yo|
+-----------+-----------+

In [24]:
df.filter(F.regexp_extract("_c0","\D+",0)=='').show()

+---+
_c0|
+---+
 1|
 2|
+---+

In [25]:
import re as r
df.filter((F.col('_c0')).isin([x for x in range(1,r.count()+1)]))


--------------------------------------------------------------------------- 
 AttributeError Traceback (most recent call last)
 <command-285087407581625> in <module> 
 1 import re as r
 ----> 2 df . filter ( ( F . col ( '_c0' ) ) . isin ( [ x for x in range ( 1 , r . count ( ) + 1 ) ] ) ) 

 AttributeError : module 're' has no attribute 'count'

In [26]:
df.filter((F.col('_c0')).isin([x for x in range(1,df.count()+1)])).show()

+---+
_c0|
+---+
 1|
 2|
+---+

In [27]:
[x for x in range(1,10)]

Out[27]: [1, 2, 3, 4, 5, 6, 7, 8, 9]

In [28]:
df.filter((F.split("_c0","")[0]).isin([x for x in range(1,10)])).show()

+---+
_c0|
+---+
 1|
 2|
+---+

In [29]:
 df.withColumn("Name", F.explode(F.array(F.col("Name"),F.col("Name"),F.col("Name"))))

In [30]:
list=[[1,'John'],
      [2,'Maria'],
      [3,'Charles']]

df=spark.createDataFrame(list,['ID','Name'])
df.show()

+---+-------+
 ID| Name|
+---+-------+
 1| John|
 2| Maria|
 3|Charles|
+---+-------+

In [31]:
df.withColumn("ID", F.explode(F.array(*[['ID']*3]))).show()

+---+-------+
 ID| Name|
+---+-------+
 1| John|
 1| John|
 1| John|
 2| Maria|
 2| Maria|
 2| Maria|
 3|Charles|
 3|Charles|
 3|Charles|
+---+-------+

In [32]:
df.withColumn("Name", F.explode(F.array(*[['Name']*3]))).show()

+---+-------+
 ID| Name|
+---+-------+
 1| John|
 1| John|
 1| John|
 2| Maria|
 2| Maria|
 2| Maria|
 3|Charles|
 3|Charles|
 3|Charles|
+---+-------+

In [33]:
stock_prices = pd.DataFrame({
        'date': ['2020-05-04', '2020-05-01', '2020-05-04', '2020-05-01', '2020-05-04', '2020-05-01'],
        'stock':['AAPL', 'AAPL', 'FB', 'FB', 'MSFT', 'MSFT'],
        'price': [290, 289, 204, 202, 177, 174]
})
df=spark.createDataFrame(stock_prices)

df.show()

+----------+-----+-----+
 date|stock|price|
+----------+-----+-----+
2020-05-04| AAPL| 290|
2020-05-01| AAPL| 289|
2020-05-04| FB| 204|
2020-05-01| FB| 202|
2020-05-04| MSFT| 177|
2020-05-01| MSFT| 174|
+----------+-----+-----+

In [34]:
list1=[['Andorra',42.506,1.5218,2,1],
      ['Australia',-31.81,115.86,1,6],
      ['Austria',41.597,12.580,4,9],
      ['Belgium',21.782,1.286,2,3],
      ['India',78.389,12.972,1,7],
      ['Ireland',9.281,9.286,9,8],
      ['USA',69.371,21.819,7,2]]

df1=spark.createDataFrame(list1,['country','Latitude','Longitude','col1','col2'])

df1.show()

list2=[['Australia',-31.81,115.86,2,6],
       ['Belgium',21.782,1.286,1,6],
       ['India',78.389,12.972,3,5],
       ['USA',69.371,21.819,2,5]]

df2=spark.createDataFrame(list2,['country','Latitude','Longitude','col1','col2'])

df2.show()

+---------+--------+---------+----+----+
 country|Latitude|Longitude|col1|col2|
+---------+--------+---------+----+----+
 Andorra| 42.506| 1.5218| 2| 1|
Australia| -31.81| 115.86| 1| 6|
 Austria| 41.597| 12.58| 4| 9|
 Belgium| 21.782| 1.286| 2| 3|
 India| 78.389| 12.972| 1| 7|
 Ireland| 9.281| 9.286| 9| 8|
 USA| 69.371| 21.819| 7| 2|
+---------+--------+---------+----+----+

+---------+--------+---------+----+----+
 country|Latitude|Longitude|col1|col2|
+---------+--------+---------+----+----+
Australia| -31.81| 115.86| 2| 6|
 Belgium| 21.782| 1.286| 1| 6|
 India| 78.389| 12.972| 3| 5|
 USA| 69.371| 21.819| 2| 5|
+---------+--------+---------+----+----+

In [35]:
df1.join(df2,['Latitude','Longitude'],'left_semi').show()

+--------+---------+---------+----+----+
Latitude|Longitude| country|col1|col2|
+--------+---------+---------+----+----+
 78.389| 12.972| India| 1| 7|
 69.371| 21.819| USA| 7| 2|
 -31.81| 115.86|Australia| 1| 6|
 21.782| 1.286| Belgium| 2| 3|
+--------+---------+---------+----+----+

In [36]:
df2.alias("t2").join(df1.alias("t1"),['Latitude','Longitude'],'left').select("t2.*").show()

+--------+---------+---------+----+----+
Latitude|Longitude| country|col1|col2|
+--------+---------+---------+----+----+
 78.389| 12.972| India| 3| 5|
 69.371| 21.819| USA| 2| 5|
 -31.81| 115.86|Australia| 2| 6|
 21.782| 1.286| Belgium| 1| 6|
+--------+---------+---------+----+----+

In [37]:
list= [['Bob1',16,['x1','x2'],['x1','x3']],
      ['Bob2',16,['x1','x2','x3'],[]]]

df=spark.createDataFrame(list,['Name','Age','P_Attribute','S_Attributes'])

df.show()


+----+---+------------+------------+
Name|Age| P_Attribute|S_Attributes|
+----+---+------------+------------+
Bob1| 16| [x1, x2]| [x1, x3]|
Bob2| 16|[x1, x2, x3]| []|
+----+---+------------+------------+

In [38]:
from pyspark.sql import functions as F
df.withColumn("Attributes", F.explode(F.array_union(F.expr("""transform(P_Attribute,x-> struct(x as Attribute,1 as Type))"""),\
              F.expr("""transform(S_Attributes,x-> struct(x as Attribute,2 as Type))"""))))\
   .select("Name", "Age", "Attributes.*").show()

+----+---+---------+----+
Name|Age|Attribute|Type|
+----+---+---------+----+
Bob1| 16| x1| 1|
Bob1| 16| x2| 1|
Bob1| 16| x1| 2|
Bob1| 16| x3| 2|
Bob2| 16| x1| 1|
Bob2| 16| x2| 1|
Bob2| 16| x3| 1|
+----+---+---------+----+

In [39]:
df.withColumn("Attributes", F.array_union(F.expr("""transform(P_Attribute,x-> struct(1,x))"""),\
              F.expr("""transform(S_Attributes,x-> struct(2,x))""")))\
   .withColumn("Attributes", F.explode("Attributes"))\
   .select("Name","Age", F.col("Attributes.x").alias("Attribute"),\
           F.col("Attributes.col1").alias("Type")).show()

+----+---+---------+----+
Name|Age|Attribute|Type|
+----+---+---------+----+
Bob1| 16| x1| 1|
Bob1| 16| x2| 1|
Bob1| 16| x1| 2|
Bob1| 16| x3| 2|
Bob2| 16| x1| 1|
Bob2| 16| x2| 1|
Bob2| 16| x3| 1|
+----+---+---------+----+

In [40]:
list=[['Bob1',16,['x1','x2'],['x1','x3'],["ab","1"],[1,2]],
      ['Bob2',16,['x1','x2','x3'],[],["a","b","c"],[]]]

df=spark.createDataFrame(list,['Name','Age','P_Attribute','S_Attributes','P_Values','S_values'])

df.show()


+----+---+------------+------------+---------+--------+
Name|Age| P_Attribute|S_Attributes| P_Values|S_values|
+----+---+------------+------------+---------+--------+
Bob1| 16| [x1, x2]| [x1, x3]| [ab, 1]| [1, 2]|
Bob2| 16|[x1, x2, x3]| []|[a, b, c]| []|
+----+---+------------+------------+---------+--------+

In [41]:
from pyspark.sql import functions as F
df.withColumn("Attributes", F.explode(F.array_union\
               (F.expr("""transform(arrays_zip(P_Attribute,P_Values),x->\
                          struct(x.P_Attribute as Attribute,1 as Type,string(x.P_Values) as Value))"""),\
                F.expr("""transform(arrays_zip(S_Attributes,S_Values),x->\
                          struct(x.S_Attributes as Attribute,2 as Type,string(x.S_Values) as Value))"""))))\
   .select("Name", "Age", "Attributes.*").show()

+----+---+---------+----+-----+
Name|Age|Attribute|Type|Value|
+----+---+---------+----+-----+
Bob1| 16| x1| 1| ab|
Bob1| 16| x2| 1| 1|
Bob1| 16| x1| 2| 1|
Bob1| 16| x3| 2| 2|
Bob2| 16| x1| 1| a|
Bob2| 16| x2| 1| b|
Bob2| 16| x3| 1| c|
+----+---+---------+----+-----+

In [42]:
from pyspark.sql import functions as F
df.withColumn("Attributes", F.explode(F.array_union\
               (F.expr("""transform(arrays_zip(P_Attribute,P_Values),x->\
                          struct(x.P_Attribute as Attribute,1 as Type,string(x.P_Values) as Value))"""),\
                F.expr("""transform(arrays_zip(S_Attributes,S_Values),x->\
                          struct(x.S_Attributes as Attribute,2 as Type,string(x.S_Values) as Value))"""))))\
   .select("Name", "Age", "Attributes.*").show()

+----+---+---------+----+-----+
Name|Age|Attribute|Type|Value|
+----+---+---------+----+-----+
Bob1| 16| x1| 1| ab|
Bob1| 16| x2| 1| 1|
Bob1| 16| x1| 2| 1|
Bob1| 16| x3| 2| 2|
Bob2| 16| x1| 1| a|
Bob2| 16| x2| 1| b|
Bob2| 16| x3| 1| c|
+----+---+---------+----+-----+

In [43]:
from pyspark.sql import functions as F
df.withColumn("Attributes", F.explode(F.array_union(F.expr("""transform(P_Attribute,x-> struct(x as Attribute,1 as Type))"""),\
              F.expr("""transform(S_Attributes,x-> struct(x as Attribute,2 as Type))"""))))\
   .select("Name", "Age", "Attributes.*").show()

In [44]:
df.withColumn("yo", F.struct(F.lit(1),F.lit(2),F.lit('yo'))).printSchema()

root
-- Name: string (nullable = true)
-- Age: long (nullable = true)
-- P_Attribute: array (nullable = true)
 |-- element: string (containsNull = true)
-- S_Attributes: array (nullable = true)
 |-- element: string (containsNull = true)
-- P_Values: array (nullable = true)
 |-- element: string (containsNull = true)
-- S_values: array (nullable = true)
 |-- element: long (containsNull = true)
-- yo: struct (nullable = false)
 |-- col1: integer (nullable = false)
 |-- col2: integer (nullable = false)
 |-- col3: string (nullable = false)

In [45]:
list= [['Bob1',16,['x1','x2'],['1','3']],
      ['Bob2',16,['x1','x2','x3'],['4','5','7']]]

df=spark.createDataFrame(list,['Name','Age','S_Attributes','S_value'])

df.show()

+----+---+------------+---------+
Name|Age|S_Attributes| S_value|
+----+---+------------+---------+
Bob1| 16| [x1, x2]| [1, 3]|
Bob2| 16|[x1, x2, x3]|[4, 5, 7]|
+----+---+------------+---------+

In [46]:
df.withColumn("yo",F.expr("""transform(arrays_zip(S_Attributes,S_value), x -> struct(x.S_Attributes as Attribute, x.S_value as Value, 2 as Type))""")).printSchema()

root
-- Name: string (nullable = true)
-- Age: long (nullable = true)
-- S_Attributes: array (nullable = true)
 |-- element: string (containsNull = true)
-- S_value: array (nullable = true)
 |-- element: string (containsNull = true)
-- yo: array (nullable = true)
 |-- element: struct (containsNull = false)
 | |-- Attribute: string (nullable = true)
 | |-- Value: string (nullable = true)
 | |-- Type: integer (nullable = false)

In [47]:
F.expr("""transform(S_Attributes,S_value, x, y -> struct(x as Attribute, y as Vaue, 2 as Type))""")))

In [48]:
df.withColumn("Attributes", F.array_union(F.expr("""transform(P_Attribute,x-> struct(x as Attribute,1 as Type))"""),\
              F.expr("""transform(S_Attributes,x-> struct(x as Attribute,2 as Type))""")))\
   .withColumn("Attributes", F.explode("Attributes"))\
   .select("Name", "Age", "Attributes.*").show()